In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages


file_path = 'Pizza_sales.xlsx' 
xls = pd.ExcelFile(file_path)
sales_data = pd.read_excel(xls, 'Sales')
pizza_data = pd.read_excel(xls, 'Pizza Data')


sales_data['order_date'] = pd.to_datetime(sales_data['order_date'])


total_revenue = sales_data['total_price'].sum()
total_pizzas_sold = sales_data['quantity'].sum()


top_selling_pizzas = sales_data.groupby('pizza_id')['quantity'].sum().reset_index()
top_selling_pizzas = top_selling_pizzas.merge(pizza_data[['pizza_id', 'pizza_name']], on='pizza_id')
top_selling_pizzas = top_selling_pizzas.sort_values('quantity', ascending=False).head(5)


sales_by_category = sales_data.merge(pizza_data[['pizza_id', 'pizza_category']], on='pizza_id')
category_sales = sales_by_category.groupby('pizza_category')['total_price'].sum().reset_index()


sales_over_time = sales_data.groupby(sales_data['order_date'].dt.to_period('M'))['total_price'].sum().reset_index()
sales_over_time['order_date'] = sales_over_time['order_date'].dt.to_timestamp()


pdf_path = 'Pizza_Sales_Dashboard.pdf'
with PdfPages(pdf_path) as pdf:
    
    plt.figure(figsize=(8, 5))
    plt.bar(['Total Revenue', 'Total Pizzas Sold'], [total_revenue, total_pizzas_sold], color=['blue', 'orange'])
    plt.title('Total Sales Overview')
    plt.ylabel('Amount')
    pdf.savefig()
    plt.close()

    
    plt.figure(figsize=(8, 5))
    sns.barplot(data=top_selling_pizzas, x='quantity', y='pizza_name', palette='Blues_d')
    plt.title('Top 5 Selling Pizzas')
    plt.xlabel('Quantity Sold')
    pdf.savefig()
    plt.close()

    
    plt.figure(figsize=(8, 5))
    sns.barplot(data=category_sales, x='total_price', y='pizza_category', palette='viridis')
    plt.title('Sales by Pizza Category')
    plt.xlabel('Total Sales ($)')
    pdf.savefig()
    plt.close()

    
    plt.figure(figsize=(8, 5))
    plt.plot(sales_over_time['order_date'], sales_over_time['total_price'], marker='o', linestyle='-')
    plt.title('Sales Over Time (Monthly)')
    plt.xlabel('Date')
    plt.ylabel('Total Sales ($)')
    plt.grid(True)
    pdf.savefig()
    plt.close()

print(f'Dashboard saved as {pdf_path}')


Dashboard saved as Pizza_Sales_Dashboard.pdf
